<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/ml801-project-NEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets

In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

def calculate_cross_entropy(model, tokenizer, sentence, translation, device):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs, labels=tokenizer(translation, return_tensors="pt", padding=True, truncation=True).input_ids.to(device))
    loss = outputs.loss.item()
    return loss

def dual_ce_filtering(corpus, model, tokenizer, threshold, device):
    model.to(device)
    filtered_corpus = []

    for sentence_lang1, sentence_lang2 in corpus:
        ce_lang1 = calculate_cross_entropy(model, tokenizer, sentence_lang1, sentence_lang2, device)
        ce_lang2 = calculate_cross_entropy(model, tokenizer, sentence_lang2, sentence_lang1, device)

        if ce_lang1 < threshold and ce_lang2 < threshold:
            filtered_corpus.append((sentence_lang1, sentence_lang2))

    return filtered_corpus

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

In [13]:
from datasets import load_dataset

dataset = load_dataset("un_multi", "ar-en")

Generating train split:   0%|          | 0/9759125 [00:00<?, ? examples/s]

In [17]:
# Sample 100,000 rows from the training set
sampled_dataset = dataset['train'].shuffle(seed=42).select(range(100000))

# Separate Arabic and English into two different columns
def separate_languages(example):
    return {
        'arabic': example['translation']['ar'],
        'english': example['translation']['en']
    }

parallel_dataset = sampled_dataset.map(separate_languages)

# Check the first few entries
print(parallel_dataset['arabic'][:5])
print(parallel_dataset['english'][:5])

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

['واضطلعت بعثة مراقبي الأمم المتحدة في طاجيكستان، وفريق الاتصال التابع للدول الضامنة، والمنظمات الدولية، والبلدان المانحة بدور لا غنى عنه في مختلف مراحل التسوية بين الطوائف الطاجيكية، التي توجت بتنفيذ اتفاق موسكو العام لإقرار السلام والوفاق الوطني في طاجيكستان.', '9 - إقرار مقررات مؤتمر الأطراف في اتفاقية فيينا.', '5- يحثّ البلدان النامية الأطراف المتأثرة، وأي بلد طرف آخر متأثر في إطار مرفق التنفيذ الإقليمي الخاص به، على مواءمة برامج عملها وأنشطتها التنفيذية الأخرى ذات الصلة بالاتفاقية مع الاستراتيجية، بأساليب تشمل تناول النتائج في إطار الأهداف التنفيذية الخمسة؛', 'وقد انخفض عدد المناطق التي نعمل فيها منذ أن قدمنا طلبنا الأولي نظرا لخيارنا المتمثل في الإبقاء على أنشطتنا في مجموعة البلدان التي تمثل موضع تركيزنا، وذلك بغية ترسيخ دورنا.', 'الممتنعون:']
['The United Nations Mission of Observers in Tajikistan, Contact Group of Guarantor States and international organizations donor countries all played an indispensable role at various stages of the inter-Tajik settlement, whose culmination w

In [22]:
parallel_dataset = parallel_dataset.map(remove_columns=['translation'])

ValueError: ignored

In [21]:
parallel_dataset

Dataset({
    features: [],
    num_rows: 0
})

In [14]:
threshold = 0.5  # Set your threshold based on your specific needs
device = "cuda" if torch.cuda.is_available() else "cpu"

filtered_dataset = dual_ce_filtering(dataset, model, tokenizer, threshold, device)
# print(filtered_dataset)

ValueError: ignored

[('This is an English sentence.', "C'est une phrase en français."),
 ('Another English sentence.', 'Une autre phrase en français.')]